In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scipy.io as sio
import anndata as ad
import seaborn as sns
import os as os
import sys as sys
sys.path.append('/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/')
from scRNA_utils import *
import operator as op
import matplotlib.colors as mcolors

# load in sc data

In [2]:
data_dir_NHDP = "/home/qiuaodon/Desktop/project_data_new/"
adata_T = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_T_cells.h5ad')
adata_1 = adata_T[adata_T[:,'PDCD1'].X > 0, :]
adata_Endo = sc.read(data_dir_NHDP + '1863-counts_cells_cohort1_Endo_cells.h5ad')
adata_2 = adata_Endo

In [3]:
colors = ["grey", "blue"]  # Start with grey and end with blue
cmap = mcolors.LinearSegmentedColormap.from_list("grey_to_blue", colors)


In [4]:
adata_1_pseudo = scRNA2PseudoBulkAnnData(adata_1, sample_id_col='sample_id')
adata_2_pseudo = scRNA2PseudoBulkAnnData(adata_2, sample_id_col='sample_id')

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [5]:
# because some samples are missing, we need to add them to the dataset
# to make sure nosamplefiltered
from scRNA_utils import add_missing_samples
import anndata
# Extract sample_ids
sample_ids_1 = set(adata_1_pseudo.obs['sample_id'])
sample_ids_2 = set(adata_2_pseudo.obs['sample_id'])

# Identify missing sample_ids
missing_sample_ids_1 = sample_ids_2 - sample_ids_1
missing_sample_ids_2 = sample_ids_1 - sample_ids_2

# Add missing sample_ids to each dataset
adata_1_pseudo = add_missing_samples(adata_1_pseudo, missing_sample_ids_1)
adata_2_pseudo = add_missing_samples(adata_2_pseudo, missing_sample_ids_2)
# Sort the obs DataFrame by sample_id in both AnnData objects
adata_1_pseudo = adata_1_pseudo[adata_1_pseudo.obs.sort_values('sample_id').index]
adata_2_pseudo = adata_2_pseudo[adata_2_pseudo.obs.sort_values('sample_id').index]

# Verify that both datasets have the same sample_ids and in the same order
assert (adata_1_pseudo.obs['sample_id'].values == adata_2_pseudo.obs['sample_id'].values).all()

# Now adata_T_pseudo and adata_Endo_pseudo have the same sample_ids in the same order

/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(
/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/merge.py:1111: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


# prepare data matrix for IVtest

In [6]:
DEG_1 = paird_ttest(adata_1, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')
DEG_2 = paird_ttest(adata_2, condition_key = 'timepoint', sample_id_col = 'sample_id', patient_id_col = 'patient_id')

Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Input adata is not pseudo-bulk RNA data. Convert to pseudo-bulk RNA data.


/home/qiuaodon/anaconda3/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [7]:
DEG_1 = DEG_1[DEG_1['pval'] < 0.05]
DEG_2 = DEG_2[DEG_2['pval'] < 0.05]
gene_1 = DEG_1.index.tolist()
gene_2 = DEG_2.index.tolist()

# Filter out invalid gene names
gene_1 = [gene for gene in gene_1 if gene in adata_1_pseudo.var_names]
gene_2 = [gene for gene in gene_2 if gene in adata_2_pseudo.var_names]

gene_1_matrix = adata_1_pseudo[:, gene_1].X
gene_2_matrix = adata_2_pseudo[:, gene_2].X
gene_1_df = pd.DataFrame(gene_1_matrix, columns=gene_1, index=adata_1_pseudo.obs['sample_id'])
gene_2_df = pd.DataFrame(gene_2_matrix, columns=gene_2, index=adata_2_pseudo.obs['sample_id'])
gene_1_df.columns = [i + '_T' for i in gene_1_df.columns]
gene_2_df.columns = [i + '_Endo' for i in gene_2_df.columns]
gene_df = gene_1_df
gene_df = pd.merge(gene_df, gene_2_df, on='sample_id')

In [8]:
gene_df['treatment'] = gene_df.index.str.contains('On').astype(int)
# get patient id from sample id remove the _On or _Pre
gene_df['patient_id'] = gene_df.index.str.replace('_On', '').str.replace('_Pre', '')
gene_df['treatment'].value_counts()

1    31
0    31
Name: treatment, dtype: int64

## group the DEGpairs together using L and R

In [9]:
results = pd.read_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_PD1vsEndo_fisherz_corr03.xlsx')
# results = results.drop(columns=['Unnamed: 0'])
# # Remove the suffix '_X' from g1 and g2
results['g1'] = results['g1'].str.replace(r'_\w+', '', regex=True)
results['g2'] = results['g2'].str.replace(r'_\w+', '', regex=True)

In [10]:
# only keep the significant results with correlation > 0.3 or < -0.3
results = results[(results['g1vsg2_correlation'] < -0.3)]

In [11]:
results

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
1513,1513,RGS16,HBEGF,CSF2,CSF2RB,0.055200,-0.300036,-0.234351
1514,1514,RGS16,HBEGF,C3,C5AR2,0.066934,-0.300036,-0.225048
1515,1515,RGS16,HBEGF,ANXA1,F2RL3,0.074350,-0.300036,-0.209734
1516,1516,RGS16,HBEGF,CCL20,S1PR4,0.059680,-0.300036,-0.232531
1517,1517,SLC2A3,TCIM,ITGAM,ICAM1,0.051008,-0.300281,-0.269544
...,...,...,...,...,...,...,...,...
3883,3883,HMGB2,ID3,LGALS3,ITGB4,0.140769,-0.428617,-0.258803
3884,3884,IL2RB,PHLDA1,ANXA1,F2RL3,0.185173,-0.429749,-0.154265
3885,3885,IL2RB,F2RL3,ANXA1,F2RL3,0.914260,-0.432298,0.024959
3886,3886,TUBB,FAM107A,ANXA1,CXCR4,0.090837,-0.442659,-0.249025


In [12]:
calculate_g1_L_correlation(results, adata_1_pseudo)
calculate_g2_R_correlation(results, adata_2_pseudo)

,Unnamed: 0,g1,g2,L,R,pValue,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,g1vsL_correlation,g2vsR_correlation
1513,1513,RGS16,HBEGF,CSF2,CSF2RB,0.055200,-0.300036,-0.234351,0.454549,-0.127081
1514,1514,RGS16,HBEGF,C3,C5AR2,0.066934,-0.300036,-0.225048,0.178552,0.151163
1515,1515,RGS16,HBEGF,ANXA1,F2RL3,0.074350,-0.300036,-0.209734,-0.100689,0.459365
1516,1516,RGS16,HBEGF,CCL20,S1PR4,0.059680,-0.300036,-0.232531,0.102383,-0.109846
1517,1517,SLC2A3,TCIM,ITGAM,ICAM1,0.051008,-0.300281,-0.269544,0.222476,-0.033694
...,...,...,...,...,...,...,...,...,...,...
3883,3883,HMGB2,ID3,LGALS3,ITGB4,0.140769,-0.428617,-0.258803,-0.199384,0.409142
3884,3884,IL2RB,PHLDA1,ANXA1,F2RL3,0.185173,-0.429749,-0.154265,-0.253159,0.395634
3885,3885,IL2RB,F2RL3,ANXA1,F2RL3,0.914260,-0.432298,0.024959,-0.253159,1.000000
3886,3886,TUBB,FAM107A,ANXA1,CXCR4,0.090837,-0.442659,-0.249025,0.418917,-0.401723


In [13]:
# remove the rows with g1vsL_correlation < 0.3 
results = results[(abs(results['g1vsL_correlation']) > 0.3) & (results['g1vsL_correlation'] < 0.99)]
# remove the rows with g2vsR_correlation < 0.3 
results = results[(abs(results['g2vsR_correlation']) > 0.3) & (results['g2vsR_correlation'] < 0.99)]

In [14]:
# get the unique (L,R) pairs in results
lr_pairs = results[['L', 'R']]
# remove the LR pairs repeated less than 3 times in the lr_pairs
lr_pairs = lr_pairs.groupby(['L', 'R']).filter(lambda x: len(x) >= 3)
lr_pairs = lr_pairs.drop_duplicates()
lr_pairs

,L,R
1520,ANXA1,LIFR
1638,HMGB2,DSG2
1640,FASLG,FAS
1642,ANXA1,F2RL3
1675,ANXA1,FAS
1683,CCL5,ACKR1
1752,AREG,EGFR
2212,ANXA1,TACR1
2254,ANXA1,MET
2628,ANXA1,APLNR


In [15]:
import pandas as pd
grouped_results = results.groupby(["L", "R"])

# Display grouped data
grouped_dict = { (ligand, receptor): group for (ligand, receptor), group in grouped_results }

# Create a new DataFrame to hold the GEM1 and GEM2 groupings
grouped_data = []
tracked_gem1 = set()
tracked_gem2 = set()

min_gene_num = 2

# Temporary list to hold data before creating DataFrame
temp_data = []

for (ligand, receptor), group in grouped_results:
    gem1 = group["g1"].drop_duplicates().tolist()
    gem2 = group["g2"].drop_duplicates().tolist()
    
    # Check the overlap between current GEM_g1 and previously tracked GEM_g1
    overlap_gem1 = len(set(gem1) & tracked_gem1)
    overlap_gem2 = len(set(gem2) & tracked_gem2)
    
    # Filter to include only those with at least three genes in GEM_g1 and GEM_g2
    # and where the overlap is below the maximum allowable threshold
    if len(gem1) > min_gene_num and len(gem2) > min_gene_num:
        temp_data.append({
            "L": ligand,
            "R": receptor,
            "g1": gem1,
            "g2": gem2,
            "Num_genes_g1": len(gem1),
            "Num_genes_g2": len(gem2)
        })
        # Update the tracked GEM_g1 and GEM_g2 sets
        tracked_gem1.update(gem1)
        tracked_gem2.update(gem2)

# Convert the temporary data to a DataFrame
grouped_df = pd.DataFrame(temp_data)

# Function to find overlapping rows
def find_overlapping_rows(df, column_name):
    overlap_info = []
    for index, row in df.iterrows():
        current_set = set(row[column_name])
        overlapping_rows = []
        for other_index, other_row in df.iterrows():
            if index != other_index:
                other_set = set(other_row[column_name])
                overlap = len(current_set & other_set)
                if overlap > len(current_set) / 3: # Set the threshold for overlap
                    overlapping_rows.append(other_index)
        overlap_info.append(overlapping_rows)
    return overlap_info

# Add columns for overlapping rows in GEM_g1 and GEM_g2
grouped_df["Overlapping_g1"] = find_overlapping_rows(grouped_df, "g1")
grouped_df["Overlapping_g2"] = find_overlapping_rows(grouped_df, "g2")


In [16]:
# rank each GEM_g1 by order of name inside
grouped_df['g1'] = grouped_df['g1'].apply(lambda x: sorted(x))
grouped_df['g2'] = grouped_df['g2'].apply(lambda x: sorted(x))
grouped_df

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2
0,ANXA1,FAS,"[NEAT1, PELI1, TNFSF14]","[AC116407.2, EZH1, GGH, ID1, PIM3, RNPC3]",3,6,"[1, 2, 3, 4]",[3]
1,ANXA1,LIFR,"[ANKRD28, NEAT1, PELI1, SLC2A3]","[IER2, PIM3, TCIM, TNFSF10]",4,4,"[0, 2, 3, 4]","[2, 3]"
2,ANXA1,MET,"[ANKRD28, NEAT1, PELI1, SLA]","[EZH1, IER2, PIM3, TNFSF10, ZBTB40]",4,5,"[0, 1, 3, 4]","[0, 1, 3]"
3,ANXA1,TACR1,"[ANKRD28, NEAT1, PELI1, TNFSF14]","[EZH1, GGH, IER2, IL6R, PIM3, TNFSF10]",4,6,"[0, 1, 2, 4]","[0, 1, 2]"
4,CCL5,APLNR,"[DDIT4, NEAT1, PELI1]","[ID1, ID3, SOX4]",3,3,"[0, 1, 2, 3]",[]
5,FASLG,FAS,"[CCNH, IL7R, NFKBIA, PRDM1, TXNIP]","[AC116407.2, BBS9, TNFSF10]",5,3,[],[]
6,HMGB2,DSG2,"[CEMIP2, DUSP2, ZFP36L1]","[DUSP4, MT1M, PDK4]",3,3,[],[]


In [17]:
# remove _0 and _1 in index of adata_2_pseudo
adata_2_pseudo.obs.index = adata_2_pseudo.obs.index.str.replace('-0', '').str.replace('-1', '')
adata_1_pseudo.obs.index = adata_1_pseudo.obs.index.str.replace('-0', '').str.replace('-1', '')

In [18]:
#calculate the correlation between g1 and g2
calculate_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)
#calculate the residuals correlation between g1, g2 and LR
calculate_residuals_GEM_correlation(grouped_df, adata_1_pseudo, adata_2_pseudo)

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,ANXA1,FAS,"[NEAT1, PELI1, TNFSF14]","[AC116407.2, EZH1, GGH, ID1, PIM3, RNPC3]",3,6,"[1, 2, 3, 4]",[3],-0.320668,-0.261797
1,ANXA1,LIFR,"[ANKRD28, NEAT1, PELI1, SLC2A3]","[IER2, PIM3, TCIM, TNFSF10]",4,4,"[0, 2, 3, 4]","[2, 3]",-0.297742,-0.289020
2,ANXA1,MET,"[ANKRD28, NEAT1, PELI1, SLA]","[EZH1, IER2, PIM3, TNFSF10, ZBTB40]",4,5,"[0, 1, 3, 4]","[0, 1, 3]",-0.589387,-0.420333
3,ANXA1,TACR1,"[ANKRD28, NEAT1, PELI1, TNFSF14]","[EZH1, GGH, IER2, IL6R, PIM3, TNFSF10]",4,6,"[0, 1, 2, 4]","[0, 1, 2]",-0.498554,-0.395009
4,CCL5,APLNR,"[DDIT4, NEAT1, PELI1]","[ID1, ID3, SOX4]",3,3,"[0, 1, 2, 3]",[],-0.580469,-0.414234
5,FASLG,FAS,"[CCNH, IL7R, NFKBIA, PRDM1, TXNIP]","[AC116407.2, BBS9, TNFSF10]",5,3,[],[],-0.268817,-0.079465
6,HMGB2,DSG2,"[CEMIP2, DUSP2, ZFP36L1]","[DUSP4, MT1M, PDK4]",3,3,[],[],0.261701,0.102450


In [19]:
grouped_df = grouped_df[grouped_df['g1vsg2_correlation'] < -0.3]

In [20]:
grouped_df

,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation
0,ANXA1,FAS,"[NEAT1, PELI1, TNFSF14]","[AC116407.2, EZH1, GGH, ID1, PIM3, RNPC3]",3,6,"[1, 2, 3, 4]",[3],-0.320668,-0.261797
2,ANXA1,MET,"[ANKRD28, NEAT1, PELI1, SLA]","[EZH1, IER2, PIM3, TNFSF10, ZBTB40]",4,5,"[0, 1, 3, 4]","[0, 1, 3]",-0.589387,-0.420333
3,ANXA1,TACR1,"[ANKRD28, NEAT1, PELI1, TNFSF14]","[EZH1, GGH, IER2, IL6R, PIM3, TNFSF10]",4,6,"[0, 1, 2, 4]","[0, 1, 2]",-0.498554,-0.395009
4,CCL5,APLNR,"[DDIT4, NEAT1, PELI1]","[ID1, ID3, SOX4]",3,3,"[0, 1, 2, 3]",[],-0.580469,-0.414234


In [21]:
grouped_df['GEM_1'] = 'GEM_' + grouped_df.index.astype(str) + '_T'
grouped_df['GEM_2'] = 'GEM_' + grouped_df.index.astype(str) + '_Endo'

/tmp/ipykernel_1602596/1871681771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_df['GEM_1'] = 'GEM_' + grouped_df.index.astype(str) + '_T'
/tmp/ipykernel_1602596/1871681771.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_df['GEM_2'] = 'GEM_' + grouped_df.index.astype(str) + '_Endo'


In [22]:
# Define a function to append '_T' to each gene name
def append_suffix(gene_list, suffix='_T'):
    # Ensure it's a list and not NaN or another unexpected type
    if isinstance(gene_list, list):
        return [gene + suffix for gene in gene_list]
    else:
        print("Unexpected format:", gene_list)
        return gene_list

# Apply the function to the GEM_g1 column to add '_T' to each gene name
DEG_pairs_T = grouped_df.copy()
DEG_pairs_T['g1'] = grouped_df['g1'].apply(lambda x: append_suffix(x, '_T'))

# Similarly for GEM_g2 if needed
DEG_pairs_T['g2'] = grouped_df['g2'].apply(lambda x: append_suffix(x, '_Endo'))
# remove the genes not in gene_df from each row in DEG_pairs_T but keep the row
DEG_pairs_T['g1'] = DEG_pairs_T['g1'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])
DEG_pairs_T['g2'] = DEG_pairs_T['g2'].apply(lambda x: [gene for gene in x if gene in gene_df.columns])


# add GEM_1_T, GEM_2_T and so on to gene_df
for i, row in DEG_pairs_T.iterrows():
    GEM_1 = DEG_pairs_T.loc[i, 'g1']
    GEM_2 = DEG_pairs_T.loc[i, 'g2']
    gene_df['GEM_' + str(i) + '_T'] = gene_df[GEM_1].mean(axis=1)
    gene_df['GEM_' + str(i) + '_Endo'] = gene_df[GEM_2].mean(axis=1)
gene_df

,AC004687.1_T,AC004817.3_T,AC004951.1_T,AC006369.1_T,AC007038.2_T,AC015813.1_T,AC016831.5_T,AC021188.1_T,AC023157.3_T,AC025164.1_T,...,treatment,patient_id,GEM_0_T,GEM_0_Endo,GEM_2_T,GEM_2_Endo,GEM_3_T,GEM_3_Endo,GEM_4_T,GEM_4_Endo
sample_id,,,,,,,,,,,,,,,,,,,,,
BIOKEY_10_On,309.215912,189.972443,46.985409,88.311661,13.239152,40.373478,43.434666,111.079369,77.687202,379.605682,...,1,BIOKEY_10,1334.541016,653.753856,1864.534424,905.898636,1228.951538,780.736795,2310.729004,1810.239624
BIOKEY_10_Pre,563.980347,44.845234,55.078514,81.338425,5.761608,25.467459,9.838195,82.176804,36.126411,164.164597,...,0,BIOKEY_10,1013.675598,821.533178,1091.872681,1014.243451,839.256348,834.343305,1589.950195,2224.432170
BIOKEY_11_On,268.598755,148.075867,37.323822,188.380936,10.964890,28.348480,45.273457,68.983269,41.890564,551.513916,...,1,BIOKEY_11,1176.213013,762.106158,1524.351074,946.426093,1000.185120,843.391535,2121.239014,2323.505941
BIOKEY_11_Pre,391.792786,26.553766,51.684788,149.571411,0.000000,17.950109,23.245640,79.012733,89.163200,415.800201,...,0,BIOKEY_11,808.864319,1151.228022,868.642944,1484.467090,634.657715,1276.841326,1483.498657,2141.467570
BIOKEY_12_On,329.073425,308.417725,18.504068,184.849304,5.138805,24.086304,120.066422,34.435535,70.810493,540.817444,...,1,BIOKEY_12,1225.527222,544.953356,1562.901611,885.268158,1047.257446,778.457279,2197.615967,1749.500692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIOKEY_7_Pre,420.561005,0.000000,0.000000,1083.950928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,BIOKEY_7,597.556091,844.841110,165.847504,1533.789746,448.167084,1308.071640,666.042419,2326.401367
BIOKEY_8_On,269.902893,409.661530,45.042774,241.828293,0.000000,0.000000,82.949020,111.544533,60.184547,431.229431,...,1,BIOKEY_8,1059.437622,680.022751,1323.179932,794.873877,858.850891,662.394897,2228.649902,2052.824788
BIOKEY_8_Pre,850.882141,0.000000,0.000000,193.594498,0.000000,0.000000,236.877213,0.000000,0.000000,245.685928,...,0,BIOKEY_8,977.724304,1101.953448,1155.033569,1348.433798,814.677368,1157.206395,2079.415771,2584.062174


In [23]:
# add GEM_1_T, GEM_2_T and so on to adata
DEG_pairs = grouped_df
for i, row in DEG_pairs.iterrows():
    GEM_1 = DEG_pairs.loc[i, 'g1']
    GEM_2 = DEG_pairs.loc[i, 'g2']
    # add GEMs to adata_1_pseudo and adata_2_pseudo
    adata_1_pseudo.obs['GEM_' + str(i) + '_T'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
    adata_2_pseudo.obs['GEM_' + str(i) + '_Endo'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
    # add GEMs to adata_1 and adata_2
    adata_1.obs['GEM_' + str(i) + '_T'] = adata_1[:, GEM_1].X.mean(axis=1)
    adata_2.obs['GEM_' + str(i) + '_Endo'] = adata_2[:, GEM_2].X.mean(axis=1)

/tmp/ipykernel_1602596/2359226253.py:7: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1_pseudo.obs['GEM_' + str(i) + '_T'] = adata_1_pseudo[:, GEM_1].X.mean(axis=1)
/tmp/ipykernel_1602596/2359226253.py:8: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_2_pseudo.obs['GEM_' + str(i) + '_Endo'] = adata_2_pseudo[:, GEM_2].X.mean(axis=1)
/tmp/ipykernel_1602596/2359226253.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1.obs['GEM_' + str(i) + '_T'] = adata_1[:, GEM_1].X.mean(axis=1)


In [24]:
CIT_GEM_test(grouped_df, adata_1_pseudo, adata_2_pseudo, gene_df, method="fisherz")

/home/qiuaodon/Desktop/PanCancer_scRNA_analysis/utils/scRNA_utils.py:1650: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_df['pValue'] = np.nan


,L,R,g1,g2,Num_genes_g1,Num_genes_g2,Overlapping_g1,Overlapping_g2,g1vsg2_correlation,g1_residualvsg2_residuals_correlation,GEM_1,GEM_2,pValue
0,ANXA1,FAS,"[NEAT1, PELI1, TNFSF14]","[AC116407.2, EZH1, GGH, ID1, PIM3, RNPC3]",3,6,"[1, 2, 3, 4]",[3],-0.320668,-0.261797,GEM_0_T,GEM_0_Endo,0.041221
2,ANXA1,MET,"[ANKRD28, NEAT1, PELI1, SLA]","[EZH1, IER2, PIM3, TNFSF10, ZBTB40]",4,5,"[0, 1, 3, 4]","[0, 1, 3]",-0.589387,-0.420333,GEM_2_T,GEM_2_Endo,0.000643
3,ANXA1,TACR1,"[ANKRD28, NEAT1, PELI1, TNFSF14]","[EZH1, GGH, IER2, IL6R, PIM3, TNFSF10]",4,6,"[0, 1, 2, 4]","[0, 1, 2]",-0.498554,-0.395009,GEM_3_T,GEM_3_Endo,0.001466
4,CCL5,APLNR,"[DDIT4, NEAT1, PELI1]","[ID1, ID3, SOX4]",3,3,"[0, 1, 2, 3]",[],-0.580469,-0.414234,GEM_4_T,GEM_4_Endo,0.000790


In [25]:
# save the grouped_df to excel
grouped_df.to_excel('/home/qiuaodon/Desktop/project_data_new/CIT_results_PD1vsEndo_grouped_0920_negative.xlsx')